### Import libraries

In [228]:
from model import climate_model, losses, dot_prod_attention
from data import data_generation, data_combine, batch_creator, gp_kernels
from keras.callbacks import ModelCheckpoint
from helpers import helpers, masks
from inference import infer
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow as tf
import numpy as np
import matplotlib 
import time
import keras
np.random.seed(1)

In [216]:
save_dir = '/Users/omernivron/Downloads/GPT_climate'

In [187]:
temp, t, token = data_combine.climate_data_to_model_input('./data/t2m_monthly_averaged_ensemble_members_1989_2019.csv')

In [190]:
## create climate train/test split

In [210]:
time_tr = t[:8000]; temp_tr = temp[:8000]; token_tr = token[:8000]
time_te = t[8000:]; temp_te = temp[8000:]; token_te = token[8000:]

In [276]:
loss_object = tf.keras.losses.MeanSquaredError()
train_loss = tf.keras.metrics.Mean(name='train_loss')
test_loss = tf.keras.metrics.Mean(name='test_loss')
m_tr = tf.keras.metrics.Mean()
m_te = tf.keras.metrics.Mean()

In [277]:
@tf.function
def train_step(decoder, optimizer_c, token_pos, time_pos, tar, pos_mask):
    '''
    A typical train step function for TF2. Elements which we wish to track their gradient
    has to be inside the GradientTape() clause. see (1) https://www.tensorflow.org/guide/migrate 
    (2) https://www.tensorflow.org/tutorials/quickstart/advanced
    ------------------
    Parameters:
    pos (np array): array of positions (x values) - the 1st/2nd output from data_generator_for_gp_mimick_gpt
    tar (np array): array of targets. Notice that if dealing with sequnces, we typically want to have the targets go from 0 to n-1. The 3rd/4th output from data_generator_for_gp_mimick_gpt  
    pos_mask (np array): see description in position_mask function
    ------------------    
    '''
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]
    combined_mask_tar = masks.create_masks(tar_inp)
    with tf.GradientTape(persistent=True) as tape:
        pred, pred_sig = decoder(token_pos, time_pos, tar_inp, True, pos_mask, combined_mask_tar)
#         print('pred: ')
#         tf.print(pred_sig)

        loss, mse, mask = losses.loss_function(tar_real, pred, pred_sig)


    gradients = tape.gradient(loss, decoder.trainable_variables)
#     tf.print(gradients)
# Ask the optimizer to apply the processed gradients.
    optimizer_c.apply_gradients(zip(gradients, decoder.trainable_variables))
    train_loss(loss)
    m_tr.update_state(mse, mask)
#     b = decoder.trainable_weights[0]
#     tf.print(tf.reduce_mean(b))
    return tar_inp, tar_real, pred, pred_sig, mask

In [278]:
@tf.function
def test_step(decoder, token_pos_te, time_pos_te, tar_te, pos_mask_te):
    '''
    
    ---------------
    Parameters:
    pos (np array): array of positions (x values) - the 1st/2nd output from data_generator_for_gp_mimick_gpt
    tar (np array): array of targets. Notice that if dealing with sequnces, we typically want to have the targets go from 0 to n-1. The 3rd/4th output from data_generator_for_gp_mimick_gpt  
    pos_mask_te (np array): see description in position_mask function
    ---------------
    
    '''
    tar_inp_te = tar_te[:, :-1]
    tar_real_te = tar_te[:, 1:]
    combined_mask_tar_te = masks.create_masks(tar_inp_te)
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    pred_te, pred_sig_te = decoder(token_pos_te, time_pos_te, tar_inp_te, False, pos_mask_te, combined_mask_tar_te)
    t_loss, t_mse, t_mask = losses.loss_function(tar_real_te, pred_te, pred_sig_te)
    test_loss(t_loss)
    m_te.update_state(t_mse, t_mask)
    return tar_real_te, pred_te, pred_sig_te, t_mask

In [279]:
tf.keras.backend.set_floatx('float64')

In [280]:
if __name__ == '__main__':
    writer = tf.summary.create_file_writer(save_dir + '/logs/')
    optimizer_c = tf.keras.optimizers.Adam(0.1)
    decoder = climate_model.Decoder(16)
    EPOCHS = 500
    batch_s  = 32
    run = 0; step = 0
    num_batches = int(temp_tr.shape[0] / batch_s)
    tf.random.set_seed(1)
    ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer = optimizer_c, net = decoder)
    main_folder = "/Users/omernivron/Downloads/GPT_climate/ckpt/check_"
    folder = main_folder + str(run); helpers.mkdir(folder)
    #https://www.tensorflow.org/guide/checkpoint
    manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=3)
    ckpt.restore(manager.latest_checkpoint)
    if manager.latest_checkpoint:
        print("Restored from {}".format(manager.latest_checkpoint))
    else:
        print("Initializing from scratch.")

    with writer.as_default():
        for epoch in range(EPOCHS):
            start = time.time()

            for batch_n in range(num_batches):
                m_tr.reset_states(); train_loss.reset_states()
                m_te.reset_states(); test_loss.reset_states()
                batch_tok_pos_tr, batch_tim_pos_tr, batch_tar_tr, _ = batch_creator.create_batch_foxes(token_tr, time_tr, temp_tr, batch_s=32)
                # batch_tar_tr shape := 128 X 59 = (batch_size, max_seq_len)
                # batch_pos_tr shape := 128 X 59 = (batch_size, max_seq_len)
                batch_pos_mask = masks.position_mask(batch_tok_pos_tr)
                tar_inp, tar_real, pred, pred_sig, mask = train_step(decoder, optimizer_c, batch_tok_pos_tr, batch_tim_pos_tr, batch_tar_tr, batch_pos_mask)

                if batch_n % 100 == 0:
                    batch_tok_pos_te, batch_tim_pos_te, batch_tar_te, _ = batch_creator.create_batch_foxes(token_te, time_te, temp_te, batch_s= 32)
                    batch_pos_mask_te = masks.position_mask(batch_tok_pos_te)
                    tar_real_te, pred_te, pred_sig_te, t_mask = test_step(decoder, batch_tok_pos_te, batch_tim_pos_te, batch_tar_te, batch_pos_mask_te)
                    helpers.print_progress(epoch, batch_n, train_loss.result(), test_loss.result(), m_tr.result())
                    helpers.tf_summaries(run, step, train_loss.result(), test_loss.result(), m_tr.result(), m_te.result())
                    manager.save()
                step += 1
                ckpt.step.assign_add(1)

            print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Already exists
Restored from /Users/omernivron/Downloads/GPT_climate/ckpt/check_0/ckpt-19573
Epoch 0 batch 0 train Loss 5.2927 test Loss 5.3505 with MSE metric 7275.1221
Epoch 0 batch 100 train Loss 5.2945 test Loss 5.3048 with MSE metric 7300.6060
Epoch 0 batch 200 train Loss 5.3605 test Loss 5.5079 with MSE metric 8308.3613
Time taken for 1 epoch: 26.369020223617554 secs

Epoch 1 batch 0 train Loss 5.3342 test Loss 5.4670 with MSE metric 7899.3184
Epoch 1 batch 100 train Loss 5.3044 test Loss 5.3164 with MSE metric 7447.8867
Epoch 1 batch 200 train Loss 5.3215 test Loss 5.5254 with MSE metric 7637.6289
Time taken for 1 epoch: 24.330953121185303 secs

Epoch 2 batch 0 train Loss 5.3616 test Loss 5.2796 with MSE metric 8343.4844
Epoch 2 batch 100 train Loss 5.3445 test Loss 5.3949 with MSE metric 8069.1621
Epoch 2 batch 200 train Loss 5.3408 test Loss 5.3492 with MSE metric 7946.4360
Time taken for 1 epoch: 24.74238109588623 secs

Epoch 3 batch 0 train Loss 5.2888 test Loss 5.2884 with 

Epoch 28 batch 200 train Loss 5.4150 test Loss 5.3162 with MSE metric 9051.6963
Time taken for 1 epoch: 23.093926906585693 secs

Epoch 29 batch 0 train Loss 5.3094 test Loss 5.4605 with MSE metric 7516.2891
Epoch 29 batch 100 train Loss 5.2739 test Loss 5.3807 with MSE metric 6956.3340
Epoch 29 batch 200 train Loss 5.2576 test Loss 5.3476 with MSE metric 6745.0264
Time taken for 1 epoch: 23.10165500640869 secs

Epoch 30 batch 0 train Loss 5.2898 test Loss 5.4235 with MSE metric 7221.7285
Epoch 30 batch 100 train Loss 5.2868 test Loss 5.3276 with MSE metric 7178.3320
Epoch 30 batch 200 train Loss 5.2720 test Loss 5.2856 with MSE metric 6977.1328
Time taken for 1 epoch: 23.20391607284546 secs

Epoch 31 batch 0 train Loss 5.2899 test Loss 5.2574 with MSE metric 7219.4038
Epoch 31 batch 100 train Loss 5.3167 test Loss 5.3384 with MSE metric 7623.7651
Epoch 31 batch 200 train Loss 5.3365 test Loss 5.3841 with MSE metric 7932.7373
Time taken for 1 epoch: 23.141672134399414 secs

Epoch 32 bat

Epoch 57 batch 100 train Loss 5.3342 test Loss 5.2860 with MSE metric 7903.9614
Epoch 57 batch 200 train Loss 5.2155 test Loss 5.3844 with MSE metric 6177.2295
Time taken for 1 epoch: 23.81431818008423 secs

Epoch 58 batch 0 train Loss 5.3856 test Loss 5.3280 with MSE metric 8585.5605
Epoch 58 batch 100 train Loss 5.3562 test Loss 5.3146 with MSE metric 8199.5322
Epoch 58 batch 200 train Loss 5.3329 test Loss 5.4534 with MSE metric 7877.0811
Time taken for 1 epoch: 23.219605922698975 secs

Epoch 59 batch 0 train Loss 5.3535 test Loss 5.3049 with MSE metric 8191.1978
Epoch 59 batch 100 train Loss 5.3660 test Loss 5.4579 with MSE metric 8412.7725
Epoch 59 batch 200 train Loss 5.2728 test Loss 5.3766 with MSE metric 6985.8491
Time taken for 1 epoch: 23.743059873580933 secs

Epoch 60 batch 0 train Loss 5.3475 test Loss 5.3626 with MSE metric 8085.1426
Epoch 60 batch 100 train Loss 5.3652 test Loss 5.4105 with MSE metric 8365.5898
Epoch 60 batch 200 train Loss 5.2795 test Loss 5.2826 with M

Epoch 86 batch 100 train Loss 5.2901 test Loss 5.4448 with MSE metric 7237.1523
Epoch 86 batch 200 train Loss 5.2517 test Loss 5.2787 with MSE metric 6608.3613
Time taken for 1 epoch: 23.277854204177856 secs

Epoch 87 batch 0 train Loss 5.2427 test Loss 5.5196 with MSE metric 6507.5483
Epoch 87 batch 100 train Loss 5.3727 test Loss 5.2662 with MSE metric 8434.9941
Epoch 87 batch 200 train Loss 5.3417 test Loss 5.3215 with MSE metric 7992.1055
Time taken for 1 epoch: 23.48769998550415 secs

Epoch 88 batch 0 train Loss 5.2513 test Loss 5.3184 with MSE metric 6648.8486
Epoch 88 batch 100 train Loss 5.3053 test Loss 5.3465 with MSE metric 7449.3945
Epoch 88 batch 200 train Loss 5.3224 test Loss 5.4345 with MSE metric 7720.5034
Time taken for 1 epoch: 23.442229986190796 secs

Epoch 89 batch 0 train Loss 5.2239 test Loss 5.3646 with MSE metric 6305.1846
Epoch 89 batch 100 train Loss 5.3448 test Loss 5.3968 with MSE metric 8065.4351
Epoch 89 batch 200 train Loss 5.3694 test Loss 5.3304 with M

Time taken for 1 epoch: 23.206897974014282 secs

Epoch 115 batch 0 train Loss 5.2154 test Loss 5.3015 with MSE metric 6104.0439
Epoch 115 batch 100 train Loss 5.2819 test Loss 5.3382 with MSE metric 7111.0859
Epoch 115 batch 200 train Loss 5.2716 test Loss 5.3270 with MSE metric 6941.3291
Time taken for 1 epoch: 23.27065372467041 secs

Epoch 116 batch 0 train Loss 5.2793 test Loss 5.2684 with MSE metric 7060.0068
Epoch 116 batch 100 train Loss 5.2640 test Loss 5.4067 with MSE metric 6846.7974
Epoch 116 batch 200 train Loss 5.2673 test Loss 5.4256 with MSE metric 6899.2773
Time taken for 1 epoch: 23.151429891586304 secs

Epoch 117 batch 0 train Loss 5.2641 test Loss 5.3882 with MSE metric 6866.2373
Epoch 117 batch 100 train Loss 5.3817 test Loss 5.3644 with MSE metric 8585.2480
Epoch 117 batch 200 train Loss 5.2342 test Loss 5.3776 with MSE metric 6416.8447
Time taken for 1 epoch: 23.17644691467285 secs

Epoch 118 batch 0 train Loss 5.2789 test Loss 5.2980 with MSE metric 7023.0371
Epoc

...

Epoch 371 batch 100 train Loss 5.1537 test Loss 5.3436 with MSE metric 5340.6006
Epoch 371 batch 200 train Loss 5.3264 test Loss 5.3698 with MSE metric 7767.5127
Time taken for 1 epoch: 25.350380182266235 secs

Epoch 372 batch 0 train Loss 5.2954 test Loss 5.4055 with MSE metric 7312.4443
Epoch 372 batch 100 train Loss 5.2692 test Loss 5.4548 with MSE metric 6921.8779
Epoch 372 batch 200 train Loss 5.3266 test Loss 5.3399 with MSE metric 7765.9302
Time taken for 1 epoch: 25.07077980041504 secs

Epoch 373 batch 0 train Loss 5.3163 test Loss 5.3610 with MSE metric 7620.5410
Epoch 373 batch 100 train Loss 5.3389 test Loss 5.4638 with MSE metric 7940.6387
Epoch 373 batch 200 train Loss 5.1829 test Loss 5.3542 with MSE metric 5669.5483
Time taken for 1 epoch: 24.41229486465454 secs

Epoch 374 batch 0 train Loss 5.4059 test Loss 5.3442 with MSE metric 8928.5938
Epoch 374 batch 100 train Loss 5.2125 test Loss 5.4817 with MSE metric 5956.4644
Epoch 374 batch 200 train Loss 5.2305 test Loss 5.4

Time taken for 1 epoch: 23.1499502658844 secs

Epoch 400 batch 0 train Loss 5.2708 test Loss 5.3669 with MSE metric 6960.3936
Epoch 400 batch 100 train Loss 5.2873 test Loss 5.4256 with MSE metric 7196.8188
Epoch 400 batch 200 train Loss 5.2844 test Loss 5.4202 with MSE metric 7153.6162
Time taken for 1 epoch: 23.256110191345215 secs

Epoch 401 batch 0 train Loss 5.2638 test Loss 5.4288 with MSE metric 6847.5273
Epoch 401 batch 100 train Loss 5.3126 test Loss 5.3164 with MSE metric 7555.6792
Epoch 401 batch 200 train Loss 5.2346 test Loss 5.2912 with MSE metric 6391.2100
Time taken for 1 epoch: 22.849128007888794 secs

Epoch 402 batch 0 train Loss 5.2644 test Loss 5.3122 with MSE metric 6857.4209
Epoch 402 batch 100 train Loss 5.2632 test Loss 5.2986 with MSE metric 6835.0308
Epoch 402 batch 200 train Loss 5.3600 test Loss 5.3494 with MSE metric 8234.2891
Time taken for 1 epoch: 21.740886926651 secs

Epoch 403 batch 0 train Loss 5.2986 test Loss 5.4309 with MSE metric 7361.7207
Epoch 4

Epoch 428 batch 100 train Loss 5.3037 test Loss 5.4034 with MSE metric 7436.4175
Epoch 428 batch 200 train Loss 5.3621 test Loss 5.4122 with MSE metric 8282.3311
Time taken for 1 epoch: 21.982819080352783 secs

Epoch 429 batch 0 train Loss 5.3030 test Loss 5.4197 with MSE metric 7404.0791
Epoch 429 batch 100 train Loss 5.3763 test Loss 5.3437 with MSE metric 8509.5859
Epoch 429 batch 200 train Loss 5.2939 test Loss 5.3400 with MSE metric 7286.3491
Time taken for 1 epoch: 21.892422199249268 secs

Epoch 430 batch 0 train Loss 5.3348 test Loss 5.3775 with MSE metric 7908.3203
Epoch 430 batch 100 train Loss 5.2188 test Loss 5.3692 with MSE metric 6053.9482
Epoch 430 batch 200 train Loss 5.2344 test Loss 5.2814 with MSE metric 6380.1357
Time taken for 1 epoch: 21.946483850479126 secs

Epoch 431 batch 0 train Loss 5.3339 test Loss 5.3367 with MSE metric 7900.1079
Epoch 431 batch 100 train Loss 5.2586 test Loss 5.3749 with MSE metric 6719.3960
Epoch 431 batch 200 train Loss 5.2638 test Loss 5

Time taken for 1 epoch: 22.623358964920044 secs

Epoch 457 batch 0 train Loss 5.2890 test Loss 5.3879 with MSE metric 7216.9902
Epoch 457 batch 100 train Loss 5.2575 test Loss 5.2655 with MSE metric 6775.1909
Epoch 457 batch 200 train Loss 5.3038 test Loss 5.3903 with MSE metric 7437.0264
Time taken for 1 epoch: 22.657464027404785 secs

Epoch 458 batch 0 train Loss 5.3164 test Loss 5.3650 with MSE metric 7616.2485
Epoch 458 batch 100 train Loss 5.3368 test Loss 5.3445 with MSE metric 7934.5234
Epoch 458 batch 200 train Loss 5.3325 test Loss 5.3318 with MSE metric 7877.6504
Time taken for 1 epoch: 22.585384845733643 secs

Epoch 459 batch 0 train Loss 5.3203 test Loss 5.4038 with MSE metric 7664.2690
Epoch 459 batch 100 train Loss 5.3001 test Loss 5.3857 with MSE metric 7362.0967
Epoch 459 batch 200 train Loss 5.2889 test Loss 5.2733 with MSE metric 7200.3452
Time taken for 1 epoch: 22.57805585861206 secs

Epoch 460 batch 0 train Loss 5.2433 test Loss 5.4268 with MSE metric 6561.9819
Epo

Epoch 485 batch 100 train Loss 5.3225 test Loss 5.3692 with MSE metric 7721.7979
Epoch 485 batch 200 train Loss 5.2944 test Loss 5.3463 with MSE metric 7300.1924
Time taken for 1 epoch: 22.402196884155273 secs

Epoch 486 batch 0 train Loss 5.3204 test Loss 5.4723 with MSE metric 7689.2246
Epoch 486 batch 100 train Loss 5.2746 test Loss 5.4246 with MSE metric 7010.4839
Epoch 486 batch 200 train Loss 5.1548 test Loss 5.3722 with MSE metric 5309.9536
Time taken for 1 epoch: 22.49403977394104 secs

Epoch 487 batch 0 train Loss 5.3312 test Loss 5.4040 with MSE metric 7845.3115
Epoch 487 batch 100 train Loss 5.3836 test Loss 5.2669 with MSE metric 8672.8652
Epoch 487 batch 200 train Loss 5.3648 test Loss 5.4243 with MSE metric 8343.5762
Time taken for 1 epoch: 22.40848207473755 secs

Epoch 488 batch 0 train Loss 5.2059 test Loss 5.2147 with MSE metric 6065.7637
Epoch 488 batch 100 train Loss 5.2934 test Loss 5.4158 with MSE metric 7270.4302
Epoch 488 batch 200 train Loss 5.3087 test Loss 5.3

In [275]:
tar_real_te[10, :]

<tf.Tensor: shape=(39,), dtype=float64, numpy=
array([277.4391 , 277.1047 , 280.83145, 277.54877, 276.43674, 280.73575,
       279.6379 , 278.9441 , 280.72787, 279.9052 , 276.1898 , 278.17764,
       276.66025, 277.8398 , 279.3794 , 277.32608, 279.0153 , 276.279  ,
       277.99994, 276.37442, 280.85577, 279.12204, 280.74197, 277.27045,
       278.04602,   0.     ,   0.     ,   0.     ,   0.     ,   0.     ,
         0.     ,   0.     ,   0.     ,   0.     ,   0.     ,   0.     ,
         0.     ,   0.     ,   0.     ])>

In [266]:
t_mask[1, :]

<tf.Tensor: shape=(39,), dtype=float64, numpy=
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.])>

In [274]:
pred_te[10, :]

<tf.Tensor: shape=(39,), dtype=float64, numpy=
array([305.5369064 , 305.53690675, 305.53690678, 305.53690751,
       305.53690744, 305.79655562, 305.79655595, 305.79655609,
       305.84506049, 305.84506066, 305.84506075, 310.90365222,
       310.90365221, 310.90365213, 105.75508275, 105.75508295,
       105.75508276, 105.75508288, 105.75508255, 105.75508251,
       105.75508224, 105.75508261, 105.75508272, 105.75508303,
       105.75508289, 105.75508286, 105.75508286, 105.75508286,
       105.75508286, 105.75508286, 105.75508286, 105.75508286,
       105.75508286, 105.75508286, 105.75508286, 105.75508286,
       105.75508286, 105.75508286, 105.75508286])>

In [273]:
((pred_te[10, :] * t_mask[10, :] ) - tar_real_te[10, :])

<tf.Tensor: shape=(39,), dtype=float64, numpy=
array([  28.0978064 ,   28.43220675,   24.70545678,   27.98813751,
         29.10016744,   25.06080562,   26.15865595,   26.85245609,
         25.11719049,   25.93986066,   29.65526075,   32.72601222,
         34.24340221,   33.06385213, -173.62431725, -171.57099705,
       -173.26021724, -170.52391712, -172.24485745, -170.61933749,
       -175.10068776, -173.36695739, -174.98688728, -171.51536697,
       -172.29093711,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ])>

In [ ]:
1 - (0.0165 / sum((tar[:, 5] - np.mean(tar[:, 5]))**2) / len(tar[:, 5]))

In [ ]:
tar - np.mean(tar, 0)

In [ ]:
tar.shape

In [ ]:
np.mean(tar[:, 0])

In [ ]:
sum((tar[:, 0] - np.mean(tar[:, 0]))**2 )/ 10000

In [ ]:
sum(sum((tar - np.mean(tar))**2)) / (tar.shape[0] * tar.shape[1])

In [ ]:
pos = df_te[560, :].reshape(1, -1)

In [ ]:
tar = df_te[561, :39].reshape(1, -1)

In [ ]:
df_te[561, :]

In [ ]:
a = inference(pos, tar, 20)

In [ ]:
with matplotlib.rc_context({'figure.figsize': [10,2.5]}):
    plt.scatter(pos[:, :39], tar[:, :39], c='black')
    plt.scatter(pos[:, 39:58], a[39:])
    plt.scatter(pos[:, 39:58], df_te[561, 39:58], c='red')

In [ ]:
# tf.data.Dataset(tf.Tensor(pad_pos_tr, value_index = 0 , dtype = tf.float32))